# Guild Awards Analysis

<ul>
<li><a href="#garothi">Garothi</a></li>
<li><a href="#enoar">Eonar</a>
    <ul>
        <li><a href="#damage-debuff">Extra Damage Debuff</a></li>
        <li><a href="#air-knock-debuff">Knocked into Air Debuff</a></li>
    </ul>
</li>
</ul>

In [1]:
import pandas as pd
import numpy as np
import requests
import warcraft_logs_fn as wl
import log_analysis as la

In [2]:
api_key = '0a60df4bb184d6c614fb96e0126bd407'

Import log details

In [3]:
df = pd.read_csv('master_list.csv', encoding="iso-8859-1")
df.head()

,log_id,log_start,log_end,pull_id,pull_start,pull_end,boss_id,boss_name,kill,player_name
0,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Petrol
1,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Radiantldeal
2,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Shallowfall
3,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Acturus
4,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Tawñ


In [8]:
df.log_id.nunique()

93

<a id='eonar'></a>
## Eonar

Spell IDs: 249016 & 249015

#### Query Tests

Find pull end time for log with Eonar fights

In [9]:
df.query('log_id == "vYDj1qQgcXWdmTnF"').pull_end.max()

10305202

Run query for debuff on targets for log

In [12]:
debuff = requests.get('https://www.warcraftlogs.com:443/v1/report/tables/debuffs/vYDj1qQgcXWdmTnF?end=10305202&by=source&abilityid=249016&api_key=' + api_key)
details = debuff.json()
details

{'auras': [{'name': 'Èllipses',
   'id': 3,
   'guid': 137327868,
   'type': 'Warlock',
   'icon': 'Warlock-Affliction',
   'totalUptime': 392648,
   'totalUses': 2,
   'bands': [{'startTime': 1587803, 'endTime': 1779949},
    {'startTime': 2285588, 'endTime': 2486090}]},
  {'name': 'Ayriea',
   'id': 17,
   'guid': 132573980,
   'type': 'Hunter',
   'icon': 'Hunter-BeastMastery',
   'totalUptime': 606958,
   'totalUses': 3,
   'bands': [{'startTime': 1472757, 'endTime': 1779949},
    {'startTime': 2186324, 'endTime': 2486090}]},
  {'name': 'Petrol',
   'id': 7,
   'guid': 110737162,
   'type': 'Mage',
   'icon': 'Mage-Fire',
   'totalUptime': 885889,
   'totalUses': 2,
   'bands': [{'startTime': 1331271, 'endTime': 1779949},
    {'startTime': 2048879, 'endTime': 2486090}]},
  {'name': 'Undertakerop',
   'id': 25,
   'guid': 157445886,
   'type': 'DeathKnight',
   'icon': 'DeathKnight-Frost',
   'totalUptime': 175560,
   'totalUses': 4,
   'bands': [{'startTime': 1687311, 'endTime': 17

In [13]:
details.keys()

dict_keys(['auras', 'useTargets', 'totalTime', 'startTime', 'endTime'])

Get name and counts

In [14]:
details['auras'][0]['name']

'Èllipses'

In [15]:
details['auras'][0]['totalUses']

2

#### Create function

In [23]:
for log in df['log_id'].unique():
    print(log)
    break

1dApmwBHyQ2vcVTF


In [26]:
df[df['log_id'] == '1dApmwBHyQ2vcVTF'].pull_end.max()

10679403

In [72]:
def buff_count(api_key, log_df, spell_id, spell_name=None, description=None, buff=True):
    df_list = []
    logs = log_df['log_id'].unique()
    for log in logs:
        print('Collecting details for log', log)
        
        # Create link
        end = log_df[log_df['log_id'] == log].pull_end.max()
        if buff is True:
            link = 'https://www.warcraftlogs.com:443/v1/report/tables/buffs/' + log + '?end=' + str(end) + \
            '&by=source&abilityid=' + str(spell_id) + '&api_key='
        else:
            link = 'https://www.warcraftlogs.com:443/v1/report/tables/debuffs/' + log + '?end=' + str(end) + \
            '&by=source&abilityid=' + str(spell_id) + '&api_key='
        
        # Complete query
        event = requests.get(link + api_key)
        details = event.json()
        
        # Get player info
        for player in details['auras']:
            name = player['name']
            print('Player added:', name)
            duration = player['totalUptime']
            df_list.append({
                'log_id': log, 
                'spell_id': spell_id,
                'spell_name': spell_name,
                'description': description,
                'player': name,
                'duration': duration
            })
    
    # Create dataframe
    df = pd.DataFrame(df_list, columns=['log_id', 'spell_id', 'spell_name', 'description', 'player', 'duration'])
    # Manage excluded spell info
    if spell_name is None:
        df.drop('spell_name', axis=1, inplace=True)
    if description is None:
        df.drop('description', axis=1, inplace=True)
    
    return df

In [11]:
test = df.query('log_id == "vYDj1qQgcXWdmTnF"')

In [18]:
extra_damage = buff_count(api_key, test, 249016, 'Feedback - Targeted', 'extra damage taken', buff=False)

Player added: Ayriea
Player added: Petrol
Player added: Èllipses
Player added: Undertakerop


In [19]:
extra_damage.head()

,log_id,debuff_id,debuff_name,debuff_description,player,debuff_duration
0,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Ayriea,606958
1,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Petrol,885889
2,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Èllipses,392648
3,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Undertakerop,175560


#### Get debuff durations

In [21]:
extra_damage = buff_count(api_key, df, 249016, 'Feedback - Targeted', 'extra damage taken', buff=False)

Player added: Tinytiki
Player added: Shadowbaine
Player added: Petrol
Player added: Ayriea
Player added: Brian
Player added: Stradivarus
Player added: Bryiah
Player added: Petrol
Player added: Verle
Player added: Illestra
Player added: Bryiah
Player added: Petrol
Player added: Ayriea
Player added: Pulsè
Player added: Thebadlock
Player added: Bryiah
Player added: Ayriea
Player added: Petrol
Player added: Radiantldeal
Player added: Brian
Player added: Èllipses
Player added: Ayriea
Player added: Shadowbaine
Player added: Kamer
Player added: Psychodruid
Player added: Thebadlock
Player added: Au
Player added: Future
Player added: Ayriea
Player added: Pulsè
Player added: Shadowbaine
Player added: Brian
Player added: Acturus
Player added: Kamer
Player added: Shadowbaine
Player added: Petrol
Player added: Tinytiki
Player added: Shallowfall
Player added: Ayriea
Player added: Petrol
Player added: Petrol
Player added: Future
Player added: Au
Player added: Yoked
Player added: Shadowbaine
Player ad

In [22]:
extra_damage.head()

,log_id,debuff_id,debuff_name,debuff_description,player,debuff_duration
0,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Tinytiki,329352
1,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Shadowbaine,439981
2,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Petrol,343314
3,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Ayriea,536246
4,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Brian,89248


In [23]:
extra_damage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
log_id                64 non-null object
debuff_id             64 non-null int64
debuff_name           64 non-null object
debuff_description    64 non-null object
player                64 non-null object
debuff_duration       64 non-null int64
dtypes: int64(2), object(4)
memory usage: 3.1+ KB


In [34]:
total_extra_damage = extra_damage.groupby('player').debuff_duration.sum().sort_values(ascending=False)
total_extra_damage = pd.DataFrame(total_extra_damage)
total_extra_damage.reset_index(inplace=True)
total_extra_damage

,player,debuff_duration
0,Petrol,10688003
1,Ayriea,8304100
2,Tinytiki,2433008
3,Èllipses,2353682
4,Shadowbaine,2088836
5,Psychodruid,1651214
6,Bryiah,1297521
7,Shallowfall,949497
8,Kamer,744238
9,Future,642201


In [27]:
knock_back = debuff_count(api_key, df, 249015, 'Feedback - Burning Embers', 'knock into air')

Player added: Kyarrix
Player added: Shadowbaine
Player added: Metonymy
Player added: Znoch
Player added: Petrol
Player added: Shadowbaine
Player added: Psychodruid
Player added: Metonymy
Player added: Brian
Player added: Brian
Player added: Shadowbaine
Player added: Kyarrix
Player added: Metonymy
Player added: Shadowbaine
Player added: Brian
Player added: Psychodruid
Player added: Metonymy
Player added: Znoch
Player added: Kyarrix
Player added: Petrol
Player added: Metonymy
Player added: Znoch
Player added: Petrol
Player added: Brianmurican
Player added: Metonymy
Player added: Metonymy
Player added: Kyarrix
Player added: Petrol
Player added: Psychodruid
Player added: Psychodruid
Player added: Metonymy
Player added: Petrol
Player added: Kyarrix
Player added: Metonymy
Player added: Psychodruid
Player added: Brianmurican
Player added: Znoch
Player added: Znoch
Player added: Kyarrix
Player added: Psychodruid
Player added: Brian
Player added: Pulsè
Player added: Èllipses
Player added: Meton

In [28]:
knock_back.head()

,log_id,debuff_id,debuff_name,debuff_description,player,debuff_duration
0,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Kyarrix,326939
1,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Shadowbaine,339102
2,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Metonymy,535104
3,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Znoch,92884
4,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Petrol,445989


In [33]:
total_knock_back = knock_back.groupby('player').debuff_duration.sum().sort_values(ascending=False)
total_knock_back = pd.DataFrame(total_knock_back)
total_knock_back.reset_index(inplace=True)
total_knock_back

,player,debuff_duration
0,Metonymy,9949922
1,Znoch,5962057
2,Shadowbaine,4539189
3,Petrol,4415304
4,Brian,3089607
5,Brianmurican,2296808
6,Psychodruid,1445497
7,Kyarrix,1258462
8,Èllipses,836739
9,Pulsè,588255


In [35]:
combined_df = pd.merge(left=total_extra_damage, right=total_knock_back, how='outer', on='player')
combined_df

,player,debuff_duration_x,debuff_duration_y
0,Petrol,10688003.0,4415304.0
1,Ayriea,8304100.0,NaN
2,Tinytiki,2433008.0,NaN
3,Èllipses,2353682.0,836739.0
4,Shadowbaine,2088836.0,4539189.0
5,Psychodruid,1651214.0,1445497.0
6,Bryiah,1297521.0,NaN
7,Shallowfall,949497.0,NaN
8,Kamer,744238.0,NaN
9,Future,642201.0,NaN


In [36]:
combined_df.rename(columns={'debuff_duration_x': 'extra_damage_duration', 
                            'debuff_duration_y': 'knock_back_duration'}, 
                   inplace=True)

In [37]:
combined_df.head()

,player,extra_damage_duration,knock_back_duration
0,Petrol,10688003.0,4415304.0
1,Ayriea,8304100.0,NaN
2,Tinytiki,2433008.0,NaN
3,Èllipses,2353682.0,836739.0
4,Shadowbaine,2088836.0,4539189.0


In [41]:
combined_df.fillna(0, inplace=True)

In [44]:
combined_df['total_duration'] = combined_df.extra_damage_duration + combined_df.knock_back_duration
combined_df.sort_values('total_duration', ascending=False, inplace=True)
combined_df

,player,extra_damage_duration,knock_back_duration,total_duration
0,Petrol,10688003.0,4415304.0,15103307.0
22,Metonymy,0.0,9949922.0,9949922.0
1,Ayriea,8304100.0,0.0,8304100.0
4,Shadowbaine,2088836.0,4539189.0,6628025.0
23,Znoch,0.0,5962057.0,5962057.0
17,Brian,238977.0,3089607.0,3328584.0
3,Èllipses,2353682.0,836739.0,3190421.0
5,Psychodruid,1651214.0,1445497.0,3096711.0
2,Tinytiki,2433008.0,0.0,2433008.0
24,Brianmurican,0.0,2296808.0,2296808.0


Total duration for Petrol (longest duration of the debuff)

In [46]:
duration = combined_df.query('player == "Petrol"').total_duration[0]
duration

15103307.0

In [50]:
duration/(60*60)/1000

4.195363055555555

Max extra_damage duration

In [61]:
id_max = combined_df.extra_damage_duration.idxmax()
combined_df.loc[id_max]

player                        Petrol
extra_damage_duration     1.0688e+07
knock_back_duration       4.4153e+06
total_duration           1.51033e+07
Name: 0, dtype: object

In [62]:
combined_df.loc[id_max].extra_damage_duration/(60*60*1000)

2.9688897222222224

Max knock_up duration

In [57]:
id_max = combined_df.knock_back_duration.idxmax()
combined_df.loc[id_max]

player                      Metonymy
extra_damage_duration              0
knock_back_duration      9.94992e+06
total_duration           9.94992e+06
Name: 22, dtype: object

In [63]:
combined_df.loc[id_max].knock_back_duration/(60*60*1000)

1.2264733333333333

In [66]:
combined_df.to_csv('guild_awards/eonar_debuff_duration.csv', encoding='iso-8859-1', index=False)

## Overall

<a id='pots'></a>
### Buff Pots

**Spell IDs**
- Prolonged Power: 229206
- Potion of Old War: 188028
- Potion of Deadly Grace: 188027

In [65]:
prolonged_power = buff_count(api_key, df, 229206, 'prolonged power')

Player added: Earrl
Player added: Shadowbaine
Player added: Brian
Player added: Radiantldeal
Player added: Ayriea
Player added: Uuglei
Player added: Shallowfall
Player added: Metonymy
Player added: Cagliostro
Player added: Petrol
Player added: Psychodruid
Player added: Tinytiki
Player added: Vaelyra
Player added: Tawñ
Player added: Divanance
Player added: Illestra
Player added: Daffy
Player added: Brianmurican
Player added: Cinzia
Player added: Thebadlock
Player added: Divanance
Player added: Ayriea
Player added: Earrl
Player added: Googboog
Player added: Znoch
Player added: Psychodruid
Player added: Elzam
Player added: Petrol
Player added: Shallowfall
Player added: Metonymy
Player added: Tinytiki
Player added: Humanraptor
Player added: Earrl
Player added: Delacram
Player added: Radiantldeal
Player added: Evayne
Player added: Vaelyra
Player added: Shadowbaine
Player added: Petrol
Player added: Severos
Player added: Oakinon
Player added: Au
Player added: Èllipses
Player added: Divanance

Player added: Ayriea
Player added: Elzam
Player added: Brianmurican
Player added: Cinzia
Player added: Divanance
Player added: Shallowfall
Player added: Metonymy
Player added: Psychodruid
Player added: Velryssa
Player added: Tinytiki
Player added: Illestra
Player added: Mythrose
Player added: Thebadlock
Player added: Znoch
Player added: Au
Player added: Petrol
Player added: Googboog
Player added: Earrl
Player added: Shadowbaine
Player added: Undertakerop
Player added: Earrl
Player added: Ayriea
Player added: Future
Player added: Metonymy
Player added: Shadowbaine
Player added: Daffy
Player added: Èllipses
Player added: Vaelyra
Player added: Petrol
Player added: Radiantldeal
Player added: Shallowfall
Player added: Psychodruid
Player added: Pulsè
Player added: Brian
Player added: Cagliostro
Player added: Divanance
Player added: Znoch
Player added: Velryssa
Player added: Shadowbaine
Player added: Ayriea
Player added: Divanance
Player added: Tinytiki
Player added: Cinzia
Player added: Bria

Player added: Brian
Player added: Kyarrix
Player added: Petrol
Player added: Au
Player added: Metonymy
Player added: Cagliostro
Player added: Earrl
Player added: Radiantldeal
Player added: Tinytiki
Player added: Èllipses
Player added: Divanance
Player added: Shadowbaine
Player added: Shallowfall
Player added: Znoch
Player added: Psychodruid
Player added: Ayriea
Player added: Psychodruid
Player added: Au
Player added: Tinytiki
Player added: Earrl
Player added: Cagliostro
Player added: Velryssa
Player added: Ayriea
Player added: Divanance
Player added: Cinzia
Player added: Shallowfall
Player added: Znoch
Player added: Shadowbaine
Player added: Googboog
Player added: Metonymy
Player added: Illestra
Player added: Brianmurican
Player added: Petrol
Player added: Radiantldeal
Player added: Cagliostro
Player added: Au
Player added: Shallowfall
Player added: Divanance
Player added: Tinytiki
Player added: Velryssa
Player added: Shadowbaine
Player added: Metonymy
Player added: Earrl
Player added:

Player added: Earrl
Player added: Au
Player added: Metonymy
Player added: Radiantldeal
Player added: Velryssa
Player added: Znoch
Player added: Petrol
Player added: Brianmurican
Player added: Cagliostro
Player added: Shallowfall
Player added: Googboog
Player added: Ayriea
Player added: Cagliostro
Player added: Illestra
Player added: Psychodruid
Player added: Radiantldeal
Player added: Tawñ
Player added: Velryssa
Player added: Ayriea
Player added: Earrl
Player added: Divanance
Player added: Shadowbaine
Player added: Shaami
Player added: Petrol
Player added: Eleint
Player added: Metonymy
Player added: Brian
Player added: Vaelyra
Player added: Velsummers
Player added: Uuglei
Player added: Ayriea
Player added: Metonymy
Player added: Earrl
Player added: Cagliostro
Player added: Divanance
Player added: Radiantldeal
Player added: Brianmurican
Player added: Au
Player added: Znoch
Player added: Velryssa
Player added: Shadowbaine
Player added: Psychodruid
Player added: Shallowfall
Player added: 

Player added: Shadowbaine
Player added: Shaami
Player added: Cagliostro
Player added: Psychodruid
Player added: Vaelyra
Player added: Uuglei
Player added: Petrol
Player added: Earrl
Player added: Divanance
Player added: Metonymy
Player added: Ayriea
Player added: Illestra
Player added: Brian
Player added: Mythrose
Player added: Radiantldeal
Player added: Velsummers
Player added: Tawñ
Player added: Daffy
Player added: Vaelyra
Player added: Èllipses
Player added: Shadowbaine
Player added: Psychodruid
Player added: Brian
Player added: Ayriea
Player added: Shallowfall
Player added: Cagliostro
Player added: Metonymy
Player added: Future
Player added: Petrol
Player added: Earrl
Player added: Velryssa
Player added: Radiantldeal
Player added: Divanance
Player added: Petrol
Player added: Tinytiki
Player added: Earrl
Player added: Ayriea
Player added: Brian
Player added: Metonymy
Player added: Shallowfall
Player added: Divanance
Player added: Psychodruid
Player added: Daffy
Player added: Illestr

In [67]:
prolonged_power.head()

,log_id,spell_id,spell_name,description,player,duration
0,1dApmwBHyQ2vcVTF,229206,prolonged power,None,Earrl,2247132
1,1dApmwBHyQ2vcVTF,229206,prolonged power,None,Shadowbaine,2652150
2,1dApmwBHyQ2vcVTF,229206,prolonged power,None,Brian,2683944
3,1dApmwBHyQ2vcVTF,229206,prolonged power,None,Radiantldeal,2505325
4,1dApmwBHyQ2vcVTF,229206,prolonged power,None,Ayriea,2671834


In [69]:
prolonged_power.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1551 entries, 0 to 1550
Data columns (total 6 columns):
log_id         1551 non-null object
spell_id       1551 non-null int64
spell_name     1551 non-null object
description    0 non-null object
player         1551 non-null object
duration       1551 non-null int64
dtypes: int64(2), object(4)
memory usage: 72.8+ KB


In [70]:
prolonged_power.drop('description', axis=1, inplace=True)

In [73]:
old_war = buff_count(api_key, df, 188028, 'old war')

Player added: Uuglei
Player added: Cagliostro
Player added: Velryssa
Player added: Vaelyra
Player added: Stradivarus
Player added: Kamer
Player added: Velryssa
Player added: Cagliostro
Player added: Stradivarus
Player added: Acturus
Player added: Velryssa
Player added: Uuglei
Player added: Vaelyra
Player added: Stradivarus
Player added: Velryssa
Player added: Cagliostro
Player added: Bryiah
Player added: Stradivarus
Player added: Velryssa
Player added: Stradivarus
Player added: Bryiah
Player added: Velryssa
Player added: Kamer
Player added: Vaelyra
Player added: Uuglei
Player added: Cagliostro
Player added: Kamer
Player added: Velryssa
Player added: Kamer
Player added: Bryiah
Player added: Velryssa
Player added: Velryssa
Player added: Vaelyra
Player added: Cagliostro
Player added: Kamer
Player added: Bryiah
Player added: Velryssa
Player added: Vaelyra
Player added: Au
Player added: Stradivarus
Player added: Kamer
Player added: Velryssa
Player added: Acturus
Player added: Vaelyra
Player

Player added: Kamer
Player added: Bryiah
Player added: Velryssa
Player added: Vaelyra
Player added: Au
Player added: Kamer
Player added: Velryssa
Player added: Bryiah
Player added: Cagliostro
Player added: Velryssa
Player added: Cagliostro
Player added: Velryssa
Player added: Bryiah
Player added: Au
Player added: Vaelyra
Player added: Kamer
Player added: Velryssa
Player added: Bryiah
Player added: Kamer
Player added: Cagliostro
Player added: Vaelyra
Player added: Uuglei
Player added: Velryssa
Player added: Cagliostro
Player added: Kamer
Player added: Uuglei
Player added: Cagliostro
Player added: Velryssa
Player added: Au
Player added: Vaelyra
Player added: Stradivarus
Player added: Cagliostro
Player added: Acturus
Player added: Velryssa
Player added: Stradivarus
Player added: Cagliostro
Player added: Stradivarus
Player added: Uuglei
Player added: Velryssa
Player added: Kamer
Player added: Stradivarus
Player added: Velryssa
Player added: Kamer
Player added: Stradivarus
Player added: Vae

In [74]:
old_war.head()

,log_id,spell_id,spell_name,player,duration
0,1dApmwBHyQ2vcVTF,188028,old war,Uuglei,640546
1,1dApmwBHyQ2vcVTF,188028,old war,Cagliostro,247335
2,1dApmwBHyQ2vcVTF,188028,old war,Velryssa,591972
3,1dApmwBHyQ2vcVTF,188028,old war,Vaelyra,259983
4,1dApmwBHyQ2vcVTF,188028,old war,Stradivarus,1109824


In [75]:
old_war.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 5 columns):
log_id        316 non-null object
spell_id      316 non-null int64
spell_name    316 non-null object
player        316 non-null object
duration      316 non-null int64
dtypes: int64(2), object(3)
memory usage: 12.4+ KB


In [76]:
deadly_grace = buff_count(api_key, df, 188027, 'deadly grace')

Player added: Shadowbaine


In [77]:
deadly_grace.head()

,log_id,spell_id,spell_name,player,duration
0,Wc18VDX9wQ62hTjy,188027,deadly grace,Shadowbaine,24918


In [78]:
pots_df = pd.concat([prolonged_power, old_war, deadly_grace])
pots_df.head()

,log_id,spell_id,spell_name,player,duration
0,1dApmwBHyQ2vcVTF,229206,prolonged power,Earrl,2247132
1,1dApmwBHyQ2vcVTF,229206,prolonged power,Shadowbaine,2652150
2,1dApmwBHyQ2vcVTF,229206,prolonged power,Brian,2683944
3,1dApmwBHyQ2vcVTF,229206,prolonged power,Radiantldeal,2505325
4,1dApmwBHyQ2vcVTF,229206,prolonged power,Ayriea,2671834


In [80]:
pots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1868 entries, 0 to 0
Data columns (total 5 columns):
log_id        1868 non-null object
spell_id      1868 non-null int64
spell_name    1868 non-null object
player        1868 non-null object
duration      1868 non-null int64
dtypes: int64(2), object(3)
memory usage: 87.6+ KB


In [81]:
pots_df.nunique()

log_id          93
spell_id         3
spell_name       3
player          54
duration      1861
dtype: int64

In [90]:
player_pots = pots_df.groupby(['player', 'spell_name']).duration.sum()
player_pots

player        spell_name     
Acturus       old war              3220369
Aqurn         prolonged power      1151747
Arutemisu     prolonged power       416614
Au            old war              3942000
              prolonged power     53950237
Ayriea        prolonged power    133517608
Brian         prolonged power    127707370
Brianmurican  prolonged power     24377004
Bryiah        old war             11304041
Cagliostro    old war             19741279
              prolonged power     63148975
Chomskey      prolonged power       258220
Cinzia        prolonged power     10719571
Daffy         prolonged power     20655756
Delacram      prolonged power       268736
Demonhoney    prolonged power      2292637
Divanance     prolonged power    119127410
Earrl         prolonged power    115989819
Eleint        prolonged power     25855211
Elzam         old war               663765
              prolonged power      6646273
Evayne        prolonged power       118127
Future        prolonged 

In [93]:
player_pots = pd.DataFrame(player_pots)
player_pots.reset_index(inplace=True)

In [94]:
player_pots.head()

,player,spell_name,duration
0,Acturus,old war,3220369
1,Aqurn,prolonged power,1151747
2,Arutemisu,prolonged power,416614
3,Au,old war,3942000
4,Au,prolonged power,53950237


In [95]:
player_pots.to_csv('guild_awards/pot_duration.csv', encoding='iso-8859-1', index=False)

Need to create a function to change alt names to final names

In [96]:
player_names = pd.read_csv('player_list.csv', encoding='iso-8859-1')
player_names.head()

,player,primary_role,alt,tier_end,tier_start,start_date
0,Vaelyra,damage,NaN,True,True,NaN
1,Metonymy,damage,NaN,True,True,NaN
2,Brian,damage,Brianmurican,True,True,NaN
3,Shaami,healer,NaN,True,False,2018-06-12
4,Petrol,damage,NaN,True,True,NaN


In [105]:
player_names[player_names['alt'] == 'Elzam'].player.iloc[0]

'Cagliostro'

In [162]:
pots_copy = player_pots.copy()

In [113]:
players = player_names['alt'].unique()

In [155]:
for index, row in pots_copy.iterrows():    
    if row['player'] in players:
        print(index, player_names[player_names['alt'] == player_name].player.iloc[0])

7 Brian
19 Brian
20 Brian
48 Brian
61 Brian


In [163]:
def change_names(df, player_names):
    players = player_names['alt'].unique()
    for index, row in df.iterrows():
        if row['player'] in players:
            df.at[index, 'player'] = player_names[player_names['alt'] == player_name].player.iloc[0]

In [164]:
change_names(pots_copy, player_names)
#pots_copy.head()

In [165]:
pots_copy.nunique()

player        50
spell_name     3
duration      63
dtype: int64

**Change names and regroup**

In [166]:
change_names(player_pots, player_names)

In [167]:
new_pots = player_pots.groupby(['player', 'spell_name']).mean().reset_index()
new_pots.head()

,player,spell_name,duration
0,Acturus,old war,3220369.0
1,Aqurn,prolonged power,1151747.0
2,Arutemisu,prolonged power,416614.0
3,Au,old war,3942000.0
4,Au,prolonged power,53950237.0


In [170]:
new_pots.head(10)

,player,spell_name,duration
0,Acturus,old war,3220369.0
1,Aqurn,prolonged power,1151747.0
2,Arutemisu,prolonged power,416614.0
3,Au,old war,3942000.0
4,Au,prolonged power,53950237.0
5,Ayriea,prolonged power,133517608.0
6,Brian,old war,663765.0
7,Brian,prolonged power,43975275.6
8,Bryiah,old war,11304041.0
9,Cagliostro,old war,19741279.0


In [172]:
new_pots.to_csv('guild_awards/pot_duration.csv', encoding='iso-8859-1', index=False)

### Healthstones

Spell ID: 6262

#### Query test

In [174]:
casts = requests.get('https://www.warcraftlogs.com:443/v1/report/tables/casts/vYDj1qQgcXWdmTnF?end=10305202&abilityid=6262&api_key=' + api_key)
details = casts.json()
details

{'entries': [{'name': 'Pulsè',
   'id': 4,
   'guid': 139894879,
   'type': 'Warlock',
   'icon': 'Warlock-Affliction',
   'total': 5,
   'activeTime': 0,
   'activeTimeReduced': 0,
   'damageAbilities': [],
   'targets': [{'name': 'Environment', 'total': 5, 'type': 'NPC'}],
   'talents': [{'name': 'Writhe in Agony',
     'guid': 196102,
     'type': 1,
     'abilityIcon': 'spell_shadow_curseofsargeras.jpg'},
    {'name': 'Absolute Corruption',
     'guid': 196103,
     'type': 1,
     'abilityIcon': 'ability_bossmannoroth_empoweredmannorothsgaze.jpg'},
    {'name': 'Demonic Circle',
     'guid': 48018,
     'type': 32,
     'abilityIcon': 'spell_shadow_demoniccirclesummon.jpg'},
    {'name': 'Howl of Terror',
     'guid': 5484,
     'type': 32,
     'abilityIcon': 'ability_warlock_howlofterror.jpg'},
    {'name': 'Soul Harvest',
     'guid': 196098,
     'type': 32,
     'abilityIcon': 'spell_warlock_demonsoul.jpg'},
    {'name': 'Sow the Seeds',
     'guid': 196226,
     'type': 1,
 

In [175]:
details.keys()

dict_keys(['entries', 'totalTime', 'logVersion'])

In [177]:
details['entries'][0]['total']

5

#### Create function

In [184]:
def cast_count(api_key, log_df, spell_id, spell_name=None, encounter_number=None):
    df_list = []
    logs = log_df['log_id'].unique()
    for log in logs:
        print('Collecting details for log', log)
        
        # Create link
        end = log_df[log_df['log_id'] == log].pull_end.max()
        if encounter_number is None:
            link = 'https://www.warcraftlogs.com:443/v1/report/tables/casts/' + log + '?end=' + str(end) + \
            '&abilityid=' + str(spell_id) + '&api_key='
        # Check if encounter id in log
        elif encounter_number not in log_df[log_df['log_id'] == log].boss_id.unique():
            continue
        else:
            link = 'https://www.warcraftlogs.com:443/v1/report/tables/casts/' + log + '?end=' + str(end) + \
            '&abilityid=' + str(spell_id) + '&encounter=' + str(encounter_number) + '&api_key='
        
        # Complete query
        event = requests.get(link + api_key)
        details = event.json()
        
        # Get player info
        for player in details['entries']:
            name = player['name']
            print('Player added:', name)
            count = player['total']
            df_list.append({
                'log_id': log, 
                'spell_id': spell_id,
                'spell_name': spell_name,
                'player': name,
                'count': count
            })
    
    # Create dataframe
    df = pd.DataFrame(df_list, columns=['log_id', 'spell_id', 'spell_name', 'player', 'count'])
    # Manage excluded spell info
    if spell_name is None:
        df.drop('spell_name', axis=1, inplace=True)
    
    return df

In [179]:
casts = cast_count(api_key, test, 6262, 'healthstone')

Player added: Petrol
Player added: Èllipses
Player added: Shadowbaine
Player added: Mythrose
Player added: Ayriea
Player added: Brian
Player added: Future
Player added: Undertakerop
Player added: Daffy
Player added: Au
Player added: Velryssa
Player added: Stradivarus
Player added: Shallowfall
Player added: Pulsè
Player added: Cagliostro
Player added: Vaelyra


In [180]:
casts

,log_id,spell_id,spell_name,player,count
0,vYDj1qQgcXWdmTnF,6262,healthstone,Petrol,1
1,vYDj1qQgcXWdmTnF,6262,healthstone,Èllipses,4
2,vYDj1qQgcXWdmTnF,6262,healthstone,Shadowbaine,7
3,vYDj1qQgcXWdmTnF,6262,healthstone,Mythrose,10
4,vYDj1qQgcXWdmTnF,6262,healthstone,Ayriea,10
5,vYDj1qQgcXWdmTnF,6262,healthstone,Brian,9
6,vYDj1qQgcXWdmTnF,6262,healthstone,Future,7
7,vYDj1qQgcXWdmTnF,6262,healthstone,Undertakerop,2
8,vYDj1qQgcXWdmTnF,6262,healthstone,Daffy,5
9,vYDj1qQgcXWdmTnF,6262,healthstone,Au,16


#### Get healthstone casts

In [182]:
casts = cast_count(api_key, df, 6262, 'healthstone')

Player added: Brian
Player added: Metonymy
Player added: Radiantldeal
Player added: Stradivarus
Player added: Mythrose
Player added: Tinytiki
Player added: Petrol
Player added: Shallowfall
Player added: Tawñ
Player added: Psychodruid
Player added: Shadowbaine
Player added: Cagliostro
Player added: Daffy
Player added: Ayriea
Player added: Mythrose
Player added: Znoch
Player added: Metonymy
Player added: Vaelyra
Player added: Kamer
Player added: Shallowfall
Player added: Thebadlock
Player added: Resta
Player added: Pulsè
Player added: Au
Player added: Divanance
Player added: Ayriea
Player added: Shadowbaine
Player added: Èllipses
Player added: Duvao
Player added: Stradivarus
Player added: Mythrose
Player added: Acturus
Player added: Future
Player added: Delacram
Player added: Oakinon
Player added: Yoked
Player added: Humanraptor
Player added: Petrol
Player added: Cagliostro
Player added: Evayne
Player added: Mythrose
Player added: Radiantldeal
Player added: Brian
Player added: Daffy
Play

Player added: Cagliostro
Player added: Tawñ
Player added: Stradivarus
Player added: Brian
Player added: Daffy
Player added: Tinytiki
Player added: Velryssa
Player added: Vaelyra
Player added: Petrol
Player added: Mythrose
Player added: Radiantldeal
Player added: Illestra
Player added: Psychodruid
Player added: Ayriea
Player added: Metonymy
Player added: Shallowfall
Player added: Shadowbaine
Player added: Au
Player added: Metonymy
Player added: Daffy
Player added: Brian
Player added: Cagliostro
Player added: Divanance
Player added: Ayriea
Player added: Earrl
Player added: Stradivarus
Player added: Mythrose
Player added: Radiantldeal
Player added: Psychodruid
Player added: Stradivarus
Player added: Brian
Player added: Earrl
Player added: Tawñ
Player added: Shallowfall
Player added: Mythrose
Player added: Daffy
Player added: Vaelyra
Player added: Ayriea
Player added: Metonymy
Player added: Shadowbaine
Player added: Cagliostro
Player added: Petrol
Player added: Velryssa
Player added: Ayrie

Player added: Future
Player added: Au
Player added: Stradivarus
Player added: Èllipses
Player added: Kamer
Player added: Acturus
Player added: Au
Player added: Brianmurican
Player added: Mythrose
Player added: Cinzia
Player added: Cagliostro
Player added: Ayriea
Player added: Radiantldeal
Player added: Metonymy
Player added: Googboog
Player added: Znoch
Player added: Shallowfall
Player added: Shadowbaine
Player added: Cagliostro
Player added: Ayriea
Player added: Future
Player added: Tinytiki
Player added: Èllipses
Player added: Mythrose
Player added: Au
Player added: Shadowbaine
Player added: Brian
Player added: Psychodruid
Player added: Kyarrix
Player added: Petrol
Player added: Shallowfall
Player added: Radiantldeal
Player added: Pulsè
Player added: Googboog
Player added: Thebadlock
Player added: Shallowfall
Player added: Psychodruid
Player added: Brianmurican
Player added: Metonymy
Player added: Ayriea
Player added: Cagliostro
Player added: Petrol
Player added: Vaelyra
Player added

Player added: Eleint
Player added: Illestra
Player added: Acturus
Player added: Brian
Player added: Shaami
Player added: Velryssa
Player added: Tawñ
Player added: Shadowbaine
Player added: Petrol
Player added: Velsummers
Player added: Shallowfall
Player added: Earrl
Player added: Mythrose
Player added: Radiantldeal
Player added: Metonymy
Player added: Divanance
Player added: Stradivarus
Player added: Kamer
Player added: Ayriea
Player added: Shadowbaine
Player added: Brian
Player added: Ayriea
Player added: Mythrose
Player added: Èllipses
Player added: Psychodruid
Player added: Cagliostro
Player added: Au
Player added: Shallowfall
Player added: Petrol
Player added: Vaelyra
Player added: Kamer
Player added: Radiantldeal
Player added: Tinytiki
Player added: Metonymy
Player added: Ayriea
Player added: Znoch
Player added: Cagliostro
Player added: Acturus
Player added: Metonymy
Player added: Bryiah
Player added: Brian
Player added: Radiantldeal
Player added: Velryssa
Player added: Au
Player 

Player added: Shadowbaine
Player added: Cagliostro
Player added: Severos
Player added: Radiantldeal
Player added: Kamer
Player added: Shallowfall
Player added: Metonymy
Player added: Vaelyra
Player added: Thebadlock
Player added: Au
Player added: Googboog
Player added: Ayriea
Player added: Brianmurican
Player added: Mohgul
Player added: Mythrose
Player added: Acturus
Player added: Znoch
Player added: Psychodruid
Player added: Metonymy
Player added: Cagliostro
Player added: Radiantldeal
Player added: Acturus
Player added: Brianmurican
Player added: Shadowbaine
Player added: Cinzia
Player added: Kamer
Player added: Psychodruid
Player added: Googboog
Player added: Ayriea
Player added: Znoch
Player added: Mythrose


In [183]:
casts.head()

,log_id,spell_id,spell_name,player,count
0,1dApmwBHyQ2vcVTF,6262,healthstone,Brian,11
1,1dApmwBHyQ2vcVTF,6262,healthstone,Metonymy,4
2,1dApmwBHyQ2vcVTF,6262,healthstone,Radiantldeal,3
3,1dApmwBHyQ2vcVTF,6262,healthstone,Stradivarus,9
4,1dApmwBHyQ2vcVTF,6262,healthstone,Mythrose,8


In [185]:
casts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334 entries, 0 to 1333
Data columns (total 5 columns):
log_id        1334 non-null object
spell_id      1334 non-null int64
spell_name    1334 non-null object
player        1334 non-null object
count         1334 non-null int64
dtypes: int64(2), object(3)
memory usage: 52.2+ KB


In [186]:
casts.nunique()

log_id        93
spell_id       1
spell_name     1
player        48
count         21
dtype: int64

In [187]:
change_names(casts, player_names)

In [188]:
casts.nunique()

log_id        93
spell_id       1
spell_name     1
player        44
count         21
dtype: int64

In [199]:
healthstone_counts = casts.groupby('player')['count'].sum().reset_index().sort_values('count', ascending=False)
healthstone_counts

,player,count
3,Brian,950
23,Mythrose,494
2,Ayriea,386
5,Cagliostro,374
1,Au,369
32,Shadowbaine,312
33,Shallowfall,308
34,Stradivarus,263
26,Psychodruid,253
28,Radiantldeal,244


In [200]:
healthstone_counts.to_csv('guild_awards/healthstones.csv', encoding='iso-8859-1', index=False)